In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
pd.set_option('display.float_format', '{:.6f}'.format)

In [121]:
df = pd.read_csv('movies_metadata.csv')
df['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [122]:
df['genres'] = df['genres'].str.replace("\'","\"")

In [123]:
df['genres'][0]

'[{"id": 16, "name": "Animation"}, {"id": 35, "name": "Comedy"}, {"id": 10751, "name": "Family"}]'

In [124]:
import json
json_cols = ['genres']

def clean_json(x):
    "Create apply function for decoding JSON"
    return json.loads(x)

# Apply the function column wise to each column of interest
for x in json_cols:
    df[x] = df[x].apply(clean_json)

In [125]:
def clean_json2(x):

    ls = []

    for y in range(len(x[0])):

        for k, v in x[0][y].items():
            # append column names to ls
            ls.append(str(k)+ "_" +str(v))
   
    for z in range(len(ls)):

        if ls[z] not in df.columns:
            df[ls[z]] = 0
            df[ls[z]].iloc[x.name] = 1
        else:
            df[ls[z]].iloc[x.name] = 1
    return

for x in json_cols:
    df[[x]].apply(clean_json2, axis=1)

In [126]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'id_16', 'name_Animation', 'id_35',
       'name_Comedy', 'id_10751', 'name_Family', 'id_12', 'name_Adventure',
       'id_14', 'name_Fantasy', 'id_10749', 'name_Romance', 'id_18',
       'name_Drama', 'id_28', 'name_Action', 'id_80', 'name_Crime', 'id_53',
       'name_Thriller', 'id_27', 'name_Horror', 'id_36', 'name_History',
       'id_878', 'name_Science Fiction', 'id_9648', 'name_Mystery', 'id_10752',
       'name_War', 'id_10769', 'name_Foreign', 'id_10402', 'name_Music',
       'id_99', 'name_Documentary', 'id_37', 'name_Western', 'id_10770',
       'name_TV Movie', 'name_Carousel Productions', 'id_11176'

In [127]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,name_Sentai Filmworks,id_33751,name_Odyssey Media,id_17161,name_Pulser Productions,id_18012,name_Rogue State,id_18013,name_The Cartel,id_23822
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,0,0,0,0,0,0,0,0,0,0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,0,0,0,0,0,0,0,0,0,0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0,0,0,0,0,0,0,0,0,0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,0,0,0,0,0,0,0,0,0,0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,0,0,0,0,0,0,0,0,0,0


In [55]:
lista1

{0: "[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",
 1: "[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",
 2: "[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",
 3: "[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",
 4: "[{'id': 35, 'name': 'Comedy'}]",
 5: "[{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': 53, 'name': 'Thriller'}]",
 6: "[{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]",
 7: "[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}]",
 8: "[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 53, 'name': 'Thriller'}]",
 9: "[{'id': 12, 'name': 'Adventure'}, {'id': 28, 'name': 'Action'}, {'id': 53, 'name': 'Thriller'}]",
 10: "[{'id': 35, 'name': 'Comedy'}, {'id

In [53]:
pd.DataFrame(eval(lista1))

TypeError: eval() arg 1 must be a string, bytes or code object

In [48]:
import json
pd.json_normalize(df['genres'])

AttributeError: 'str' object has no attribute 'values'